In [30]:
# ---------------------------------------------------------------------------------------------------------------------
# IMPORTS
# ---------------------------------------------------------------------------------------------------------------------
#TODO organize these
import datetime
from datetime import date
import seaborn as sns
import copy
# from datetime import date_range
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import os
import os.path
import configparser


# ---------------------------------------------------------------------------------------------------------------------
# CONFIGURATION and INITIALIZATION
# ---------------------------------------------------------------------------------------------------------------------

# Read in the config file
config = configparser.ConfigParser()
config.read('config.ini')
PROJ_DIR = config['DEFAULT']['PROJECT_DIR']
#PROJ_DIR = config['PROJECT_DIR']

# pandas settings for outputting numerical data
#TODO Robert, why are these set this way?  Does this make the output play nicely with Jupyter?
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Close all previous plots
plt.close("all")

#TODO Make this a command line argument
MAKE_PLOTS = False

#TODO Make this a command line argument
#TODO Check to make sure this exists
OUTPUT_DIR = os.path.join(PROJ_DIR,'outdir').replace('\'','')
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

#TODO Make this a command line argument
date_rng = pd.date_range(start='12/11/2018', end='4/30/2019', freq='D')
# date_rng = pd.date_range(start='11/25/2018', end='5/13/2019', freq='D')

# Read in the data
# RSH added replace command to get ride of single quote
DATA_FILE = os.path.join(PROJ_DIR,'BiasCorrectionData_new.csv').replace('\'','')

#TODO Auto-detect this?
COLUMNS = ['HUR2', 'MTB2', 'WAP2', 'STV2', 'SNO2', 'LVN2', 'MIS2', 'CMT2', 'PAR2', 'WHP2', 'TML2', 'MHM2', \
           'HUR3', 'MTB3', 'WAP3', 'STV3', 'SNO3', 'LVN3', 'MIS3', 'CMT3', 'PAR3', 'WHP3', 'TML3', 'MHM3']
NAMES = ['HUR', 'MTB', 'WAP', 'STV', 'SNO', 'LVN', 'MIS', 'CMT', 'PAR', 'WHP', 'TML', 'MHM']
# names = ['STV']

# ---------------------------------------------------------------------------------------------------------------------
# METHODS
# ---------------------------------------------------------------------------------------------------------------------

def plotFigure(data_plot, file_name, order):
    fig = plt.figure(order, figsize=(9, 6))
    ax = fig.add_subplot(111)
    bp = ax.boxplot(data_plot)
    fig.savefig(file_name, bbox_inches='tight')
    plt.close()

# ---------------------------------------------------------------------------------------------------------------------
# SCRIPT BODY
# ---------------------------------------------------------------------------------------------------------------------
#TODO Remove various commented out scratch code.

# Read in the data file
df = pd.read_csv(DATA_FILE)
df.columns = df.columns.str.strip()
# df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.iloc[16:157, :]
# df['WRF Day'].shape
# df.truncate(after=112)
df['Date'] = date_rng
df = df.set_index('Date')

df.drop(COLUMNS, inplace=True, axis=1)

for name in NAMES:
    tau = 30
    df2 = copy.deepcopy(df.filter(regex=name))
    df2[name + '_CF'] = 0
    df2[name + '_BC'] = 0
    bc_fcst = df2[name + '_BC']
    df2[name + '_Raw_Bias'] = 0
    raw_bias = df2[name + '_Raw_Bias']
    df2[name + '_BC_Bias'] = 0
    bc_bias = df2[name + '_BC_Bias']
    obs = df2[name + '1']
    fcst = df2[name + '4']
    cf = df2[name + '_CF']
    cf.iloc[0] = 1
    df3 = copy.deepcopy(df.filter(regex=name).dropna(axis=0, how='any'))
    df3[name + '_CF'] = 0
    cf_drop = df3[name + '_CF']
    df3[name + '_BC'] = 0
    bc_fcst_drop = df3[name + '_BC']
    df3[name + '_Raw_Bias'] = 0
    raw_bias_drop = df3[name + '_Raw_Bias']
    df3[name + '_BC_Bias'] = 0
    bc_bias_drop = df3[name + '_BC_Bias']
    obs_drop = df2[name + '1']
    fcst_drop = df2[name + '4']
    cf_drop = df2[name + '_CF']
    cf.iloc[0] = 1

    for i in range(len(df2) - 1):
        if (obs.iloc[i] <= 0.01 or np.isnan(obs.iloc[i]) == True or np.isnan(fcst.iloc[i]) == True):
            cf.iloc[i + 1] = cf.iloc[i]
        else:
            cf.iloc[i + 1] = ((tau - 1) / tau) * cf.iloc[i] + (1 / tau) * (fcst.iloc[i] / obs.iloc[i])
            # code to avoid large jumps in cf
            if (abs((cf.iloc[i + 1] / cf.iloc[i])) > 1.5 and (fcst.iloc[i] + obs.iloc[i]) < 1):
                cf.iloc[i + 1] = cf.iloc[i] + (cf.iloc[i + 1] - cf.iloc[i]) / (cf.iloc[i + 1] + cf.iloc[i])
        bc_fcst.iloc[i] = fcst.iloc[i] / cf.iloc[i]
        bc_bias.iloc[i] = bc_fcst.iloc[i] - obs.iloc[i]
        raw_bias.iloc[i] = fcst.iloc[i] - obs.iloc[i]
        #         print("date is " + str(df.index.date[i]) + "; fcst is " + str(round(fcst.iloc[i],2)) + "; obs is " + str(round(obs.iloc[i],2)) + "; cf is " + str(round(cf.iloc[i],2)) + \
        #                 "; fcst_bc is " + str(round(bc_fcst.iloc[i],2)))
        print("date is " + str(df.index.date[i]) + "; fcst is " + str(round(fcst.iloc[i], 2)) + "; obs is " + str(
            round(obs.iloc[i], 2)) + "; cf is " + str(round(cf.iloc[i], 2)) + \
              "; bc_fcst is " + str(round(bc_fcst.iloc[i], 2)))
        
        a = open((OUTPUT_DIR + '/' + name + '_precip.txt').replace('\'',''), 'w')
        a.write(str(df3))
        a.close()

    # Skip plot generation if so specified.
    if not MAKE_PLOTS:
        continue

    fig = plt.figure(figsize=(16, 16))
    raw_bias.plot(figsize=(20, 10), fontsize=20, color="green")
    bc_bias.plot(figsize=(20, 10), fontsize=20, color="red")
    #     bc_fcst.plot(figsize=(20,10), fontsize=20, color="blue")
    #     fcst.plot(figsize=(20,10), fontsize=20, color="magenta")
    #     obs.plot(figsize=(20,10), fontsize=20, color="orange")
    cf.plot(figsize=(20, 10), fontsize=20, color="black")
    plt.xlabel('Month', fontsize=20)
    plt.ylabel('Precip Bias (")', fontsize=20)
    plt.figtext(0.35, 0.85, name + " Raw 1.33-km WRF MAE = " + str(
        round(metrics.mean_absolute_error(df2[name + '4'], df2[name + '1']), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)
    plt.figtext(0.35, 0.8, name + " Raw 1.33-km WRF MSE = " + str(
        round(metrics.mean_squared_error(df2[name + '4'], df2[name + '1']), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)
    plt.figtext(0.35, 0.75, name + " Raw 1.33-km WRF RMSE = " + str(
        round(np.sqrt(metrics.mean_absolute_error(df2[name + '4'], df2[name + '1'])), 3)),
                wrap=True, horizontalalignment='center', fontsize=16)
    plt.figtext(0.65, 0.85, name + " BC 1.33-km WRF MAE = " + str(
        round(metrics.mean_absolute_error(df2[name + '_BC'], df2[name + '1']), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)
    plt.figtext(0.35, 0.15, name + " Mean Raw 1.33-km WRF Bias = " + str(
        round(raw_bias.loc['2018-12-25 00:00:00':'2019-05-13 00:00:00'].mean(), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)
    plt.figtext(0.65, 0.8, name + " BC 1.33-km WRF MSE = " + str(
        round(metrics.mean_squared_error(df2[name + '_BC'], df2[name + '1']), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)
    plt.figtext(0.65, 0.75, name + " BC 1.33-km WRF RMSE = " + str(
        round(np.sqrt(metrics.mean_absolute_error(df2[name + '_BC'], df2[name + '1'])), 3)),
                wrap=True, horizontalalignment='center', fontsize=16)
    plt.figtext(0.65, 0.15, name + " Mean BC WRF 1.33-km Bias = " + str(
        round(bc_bias.loc['2018-12-25 00:00:00':'2019-05-13 00:00:00'].mean(), 3)), wrap=True,
                horizontalalignment='center', fontsize=16)

    plt.legend(fontsize=16)
    plt.title("STN = " + name + " FH12-36 Forecast Comparison: 1.33-km WRF and BC WRF Precip Bias", fontsize=20)
    plt.show()
    fig.savefig(PROJ_DIR + '/STN=' + name + '_WRF_vs_BCWRF.png', dpi=180)
    plt.close()

/Users/robertshahn/anaconda3/envs/wrf-proj/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


date is 2018-12-11; fcst is 0.36; obs is 0.16; cf is 1.0; bc_fcst is 0.36
date is 2018-12-12; fcst is 1.07; obs is 1.25; cf is 1.04; bc_fcst is 1.03
date is 2018-12-13; fcst is 0.52; obs is 0.92; cf is 1.04; bc_fcst is 0.5
date is 2018-12-14; fcst is 0.32; obs is 0.58; cf is 1.02; bc_fcst is 0.31
date is 2018-12-15; fcst is 0.75; obs is 0.5; cf is 1.0; bc_fcst is 0.75
date is 2018-12-16; fcst is 0.22; obs is 0.27; cf is 1.02; bc_fcst is 0.22
date is 2018-12-17; fcst is 0.43; obs is 1.03; cf is 1.01; bc_fcst is 0.42
date is 2018-12-18; fcst is 0.46; obs is 1.03; cf is 0.99; bc_fcst is 0.46
date is 2018-12-19; fcst is 0.41; obs is 1.06; cf is 0.98; bc_fcst is 0.42
date is 2018-12-20; fcst is 0.18; obs is 0.63; cf is 0.96; bc_fcst is 0.19
date is 2018-12-21; fcst is 1.55; obs is 1.15; cf is 0.93; bc_fcst is 1.66
date is 2018-12-22; fcst is 0.24; obs is 0.04; cf is 0.95; bc_fcst is 0.25
date is 2018-12-23; fcst is 0.58; obs is 0.64; cf is 1.12; bc_fcst is 0.52
date is 2018-12-24; fcst is 0

date is 2019-04-06; fcst is 0.12; obs is 0.38; cf is 1.2; bc_fcst is 0.1
date is 2019-04-07; fcst is 0.36; obs is 0.91; cf is 1.18; bc_fcst is 0.31
date is 2019-04-08; fcst is 0.0; obs is 0.1; cf is 1.15; bc_fcst is 0.0
date is 2019-04-09; fcst is 0.21; obs is 0.63; cf is 1.11; bc_fcst is 0.19
date is 2019-04-10; fcst is 0.02; obs is 0.01; cf is 1.08; bc_fcst is 0.02
date is 2019-04-11; fcst is 0.39; obs is 0.23; cf is 1.08; bc_fcst is 0.36
date is 2019-04-12; fcst is 0.35; obs is 0.17; cf is 1.11; bc_fcst is 0.32
date is 2019-04-13; fcst is 0.01; obs is 0.0; cf is 1.14; bc_fcst is 0.01
date is 2019-04-14; fcst is 0.2; obs is 0.47; cf is 1.14; bc_fcst is 0.18
date is 2019-04-15; fcst is 0.2; obs is 0.06; cf is 1.11; bc_fcst is 0.18
date is 2019-04-16; fcst is 0.15; obs is 0.0; cf is 1.19; bc_fcst is 0.13
date is 2019-04-17; fcst is 0.0; obs is 0.0; cf is 1.19; bc_fcst is 0.0
date is 2019-04-18; fcst is 0.03; obs is 0.02; cf is 1.19; bc_fcst is 0.03
date is 2019-04-19; fcst is 0.07; obs

date is 2019-03-10; fcst is 0.0; obs is 0.0; cf is 1.14; bc_fcst is 0.0
date is 2019-03-11; fcst is 0.0; obs is 0.0; cf is 1.14; bc_fcst is 0.0
date is 2019-03-12; fcst is 0.31; obs is 0.53; cf is 1.14; bc_fcst is 0.27
date is 2019-03-13; fcst is 0.1; obs is 0.21; cf is 1.12; bc_fcst is 0.09
date is 2019-03-14; fcst is 0.12; obs is 0.0; cf is 1.1; bc_fcst is 0.11
date is 2019-03-15; fcst is 0.05; obs is 0.0; cf is 1.1; bc_fcst is 0.05
date is 2019-03-16; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-17; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-18; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-19; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-20; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-21; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-22; fcst is 0.0; obs is 0.0; cf is 1.1; bc_fcst is 0.0
date is 2019-03-23; fcst is 0.75; obs is 0.22; cf is 1.1; bc_fcs

date is 2019-02-17; fcst is 0.01; obs is 0.02; cf is 1.91; bc_fcst is 0.01
date is 2019-02-18; fcst is 0.0; obs is 0.0; cf is 1.87; bc_fcst is 0.0
date is 2019-02-19; fcst is 0.0; obs is 0.0; cf is 1.87; bc_fcst is 0.0
date is 2019-02-20; fcst is 0.25; obs is 0.05; cf is 1.87; bc_fcst is 0.13
date is 2019-02-21; fcst is 0.03; obs is 0.0; cf is 1.97; bc_fcst is 0.02
date is 2019-02-22; fcst is 0.0; obs is 0.0; cf is 1.97; bc_fcst is 0.0
date is 2019-02-23; fcst is 0.35; obs is 0.21; cf is 1.97; bc_fcst is 0.18
date is 2019-02-24; fcst is 0.02; obs is 0.02; cf is 1.96; bc_fcst is 0.01
date is 2019-02-25; fcst is 0.03; obs is 0.05; cf is 1.93; bc_fcst is 0.02
date is 2019-02-26; fcst is 0.0; obs is 0.0; cf is 1.88; bc_fcst is 0.0
date is 2019-02-27; fcst is 0.0; obs is 0.02; cf is 1.88; bc_fcst is 0.0
date is 2019-02-28; fcst is 0.03; obs is 0.12; cf is 1.82; bc_fcst is 0.02
date is 2019-03-01; fcst is 0.02; obs is 0.0; cf is 1.77; bc_fcst is 0.01
date is 2019-03-02; fcst is 0.0; obs is 0

...

date is 2019-03-29; fcst is 0.08; obs is 0.03; cf is 1.54; bc_fcst is 0.05
date is 2019-03-30; fcst is 0.39; obs is 0.39; cf is 1.58; bc_fcst is 0.25
date is 2019-03-31; fcst is 0.0; obs is 0.0; cf is 1.56; bc_fcst is 0.0
date is 2019-04-01; fcst is 0.0; obs is 0.0; cf is 1.56; bc_fcst is 0.0
date is 2019-04-02; fcst is 0.0; obs is 0.0; cf is 1.56; bc_fcst is 0.0
date is 2019-04-03; fcst is 0.02; obs is 0.01; cf is 1.56; bc_fcst is 0.01
date is 2019-04-04; fcst is 0.17; obs is 0.26; cf is 1.56; bc_fcst is 0.11
date is 2019-04-05; fcst is 0.15; obs is 0.12; cf is 1.53; bc_fcst is 0.1
date is 2019-04-06; fcst is 0.52; obs is 0.47; cf is 1.52; bc_fcst is 0.34
date is 2019-04-07; fcst is 1.1; obs is 1.02; cf is 1.51; bc_fcst is 0.73
date is 2019-04-08; fcst is 0.27; obs is 0.16; cf is 1.49; bc_fcst is 0.18
date is 2019-04-09; fcst is 0.48; obs is 0.81; cf is 1.5; bc_fcst is 0.32
date is 2019-04-10; fcst is 0.31; obs is 0.37; cf is 1.47; bc_fcst is 0.21
date is 2019-04-11; fcst is 0.55; obs

date is 2019-03-03; fcst is 0.0; obs is 0.0; cf is 1.16; bc_fcst is 0.0
date is 2019-03-04; fcst is 0.0; obs is 0.0; cf is 1.16; bc_fcst is 0.0
date is 2019-03-05; fcst is 0.0; obs is 0.0; cf is 1.16; bc_fcst is 0.0
date is 2019-03-06; fcst is 0.0; obs is 0.0; cf is 1.16; bc_fcst is 0.0
date is 2019-03-07; fcst is 0.4; obs is 0.51; cf is 1.16; bc_fcst is 0.35
date is 2019-03-08; fcst is 0.71; obs is 0.3; cf is 1.14; bc_fcst is 0.62
date is 2019-03-09; fcst is 0.39; obs is 0.21; cf is 1.18; bc_fcst is 0.33
date is 2019-03-10; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-03-11; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-03-12; fcst is 0.97; obs is 0.68; cf is 1.21; bc_fcst is 0.8
date is 2019-03-13; fcst is 1.5; obs is 1.28; cf is 1.21; bc_fcst is 1.24
date is 2019-03-14; fcst is 0.01; obs is 0.27; cf is 1.21; bc_fcst is 0.01
date is 2019-03-15; fcst is 0.0; obs is 0.0; cf is 1.17; bc_fcst is 0.0
date is 2019-03-16; fcst is 0.0; obs is 0.0; cf is

date is 2019-02-12; fcst is 1.59; obs is 0.5; cf is 1.81; bc_fcst is 0.88
date is 2019-02-13; fcst is 0.97; obs is 0.78; cf is 1.86; bc_fcst is 0.52
date is 2019-02-14; fcst is 0.1; obs is 0.32; cf is 1.84; bc_fcst is 0.05
date is 2019-02-15; fcst is 0.26; obs is 0.28; cf is 1.78; bc_fcst is 0.15
date is 2019-02-16; fcst is 0.26; obs is 0.04; cf is 1.76; bc_fcst is 0.15
date is 2019-02-17; fcst is 0.2; obs is 0.04; cf is 1.91; bc_fcst is 0.1
date is 2019-02-18; fcst is 0.08; obs is 0.02; cf is 2.02; bc_fcst is 0.04
date is 2019-02-19; fcst is 0.0; obs is 0.0; cf is 2.08; bc_fcst is 0.0
date is 2019-02-20; fcst is 0.72; obs is 0.35; cf is 2.08; bc_fcst is 0.35
date is 2019-02-21; fcst is 0.18; obs is 0.24; cf is 2.08; bc_fcst is 0.09
date is 2019-02-22; fcst is 0.0; obs is 0.0; cf is 2.04; bc_fcst is 0.0
date is 2019-02-23; fcst is 0.54; obs is 0.25; cf is 2.04; bc_fcst is 0.26
date is 2019-02-24; fcst is 0.16; obs is 0.21; cf is 2.04; bc_fcst is 0.08
date is 2019-02-25; fcst is 0.22; o

date is 2019-01-20; fcst is 0.19; obs is 0.52; cf is 1.27; bc_fcst is 0.15
date is 2019-01-21; fcst is 0.29; obs is 0.44; cf is 1.24; bc_fcst is 0.23
date is 2019-01-22; fcst is 0.0; obs is 0.0; cf is 1.22; bc_fcst is 0.0
date is 2019-01-23; fcst is 1.5; obs is 2.02; cf is 1.22; bc_fcst is 1.23
date is 2019-01-24; fcst is 0.72; obs is 0.53; cf is 1.21; bc_fcst is 0.6
date is 2019-01-25; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-26; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-27; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-28; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-29; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-30; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-01-31; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-02-01; fcst is 0.0; obs is 0.0; cf is 1.21; bc_fcst is 0.0
date is 2019-02-02; fcst is 0.43; obs is 0.3; cf is 1.

date is 2018-12-31; fcst is 0.89; obs is 0.75; cf is 1.25; bc_fcst is 0.71
date is 2019-01-01; fcst is 0.0; obs is 0.0; cf is 1.25; bc_fcst is 0.0
date is 2019-01-02; fcst is 0.0; obs is 0.0; cf is 1.25; bc_fcst is 0.0
date is 2019-01-03; fcst is 0.0; obs is 0.01; cf is 1.25; bc_fcst is 0.0
date is 2019-01-04; fcst is 0.04; obs is 0.22; cf is 1.25; bc_fcst is 0.03
date is 2019-01-05; fcst is 0.34; obs is 0.29; cf is 1.21; bc_fcst is 0.28
date is 2019-01-06; fcst is 0.86; obs is 0.17; cf is 1.21; bc_fcst is 0.71
date is 2019-01-07; fcst is 0.59; obs is 0.77; cf is 1.34; bc_fcst is 0.44
date is 2019-01-08; fcst is 0.71; obs is 0.15; cf is 1.32; bc_fcst is 0.54
date is 2019-01-09; fcst is 0.85; obs is 0.42; cf is 1.43; bc_fcst is 0.59
date is 2019-01-10; fcst is 0.63; obs is 0.27; cf is 1.45; bc_fcst is 0.43
date is 2019-01-11; fcst is 0.01; obs is 0.01; cf is 1.48; bc_fcst is 0.01
date is 2019-01-12; fcst is 0.0; obs is 0.0; cf is 1.48; bc_fcst is 0.0
date is 2019-01-13; fcst is 0.0; obs

In [27]:
print(OUTPUT_DIR)

/Users/robertshahn/Documents/python-projects/wx_aval_repo/BiasCorrection/outdir


In [10]:
print(DATA_FILE.strip())

'/Users/robertshahn/Documents/python-projects/wx_aval_repo/BiasCorrection'/BiasCorrectionData_new.csv


In [16]:
print(DATA_FILE.replace('\'',''))

/Users/robertshahn/Documents/python-projects/wx_aval_repo/BiasCorrection/BiasCorrectionData_new.csv


In [31]:
print(cf.iloc[i] + (cf.iloc[i + 1] - cf.iloc[i]) / (cf.iloc[i + 1] + cf.iloc[i]))

1.48487390946


In [32]:
print(cf.iloc[i])

1.48487390946


In [33]:
cf.iloc[i+1]

1.4848739094634502

In [41]:
for x2 in range(1,101,1):
    for y2 in range(1,101,1):
        x = x2/100
        y = y2/100
        ycorr = x + (y-x)/(y+x)
        print(x, y, ycorr)

0.01 0.01 0.01
0.01 0.02 0.3433333333333334
0.01 0.03 0.5099999999999999
0.01 0.04 0.61
0.01 0.05 0.6766666666666666
0.01 0.06 0.7242857142857143
0.01 0.07 0.76
0.01 0.08 0.7877777777777779
0.01 0.09 0.81
0.01 0.1 0.8281818181818182
0.01 0.11 0.8433333333333334
0.01 0.12 0.8561538461538462
0.01 0.13 0.8671428571428571
0.01 0.14 0.8766666666666666
0.01 0.15 0.8849999999999999
0.01 0.16 0.8923529411764705
0.01 0.17 0.8988888888888888
0.01 0.18 0.9047368421052631
0.01 0.19 0.9099999999999999
0.01 0.2 0.9147619047619047
0.01 0.21 0.9190909090909091
0.01 0.22 0.9230434782608695
0.01 0.23 0.9266666666666666
0.01 0.24 0.9299999999999999
0.01 0.25 0.933076923076923
0.01 0.26 0.9359259259259258
0.01 0.27 0.9385714285714285
0.01 0.28 0.9410344827586207
0.01 0.29 0.9433333333333332
0.01 0.3 0.9454838709677419
0.01 0.31 0.9475
0.01 0.32 0.9493939393939393
0.01 0.33 0.9511764705882353
0.01 0.34 0.9528571428571428
0.01 0.35 0.9544444444444444
0.01 0.36 0.955945945945946
0.01 0.37 0.9573684210526315


0.07 0.02 -0.48555555555555546
0.07 0.03 -0.33000000000000007
0.07 0.04 -0.20272727272727276
0.07 0.05 -0.09666666666666668
0.07 0.06 -0.006923076923076976
0.07 0.07 0.07
0.07 0.08 0.13666666666666663
0.07 0.09 0.19499999999999995
0.07 0.1 0.2464705882352941
0.07 0.11 0.2922222222222222
0.07 0.12 0.33315789473684204
0.07 0.13 0.37
0.07 0.14 0.4033333333333333
0.07 0.15 0.4336363636363636
0.07 0.16 0.4613043478260869
0.07 0.17 0.48666666666666664
0.07 0.18 0.51
0.07 0.19 0.5315384615384615
0.07 0.2 0.5514814814814815
0.07 0.21 0.5699999999999998
0.07 0.22 0.5872413793103448
0.07 0.23 0.6033333333333333
0.07 0.24 0.6183870967741936
0.07 0.25 0.6325000000000001
0.07 0.26 0.6457575757575758
0.07 0.27 0.658235294117647
0.07 0.28 0.6699999999999999
0.07 0.29 0.681111111111111
0.07 0.3 0.6916216216216216
0.07 0.31 0.7015789473684211
0.07 0.32 0.711025641025641
0.07 0.33 0.72
0.07 0.34 0.7285365853658536
0.07 0.35 0.7366666666666666
0.07 0.36 0.7444186046511627
0.07 0.37 0.7518181818181817
0.0

0.11 0.18 0.35137931034482756
0.11 0.19 0.37666666666666665
0.11 0.2 0.4003225806451613
0.11 0.21 0.42249999999999993
0.11 0.22 0.4433333333333333
0.11 0.23 0.46294117647058824
0.11 0.24 0.48142857142857143
0.11 0.25 0.49888888888888894
0.11 0.26 0.5154054054054055
0.11 0.27 0.5310526315789474
0.11 0.28 0.545897435897436
0.11 0.29 0.56
0.11 0.3 0.5734146341463415
0.11 0.31 0.5861904761904763
0.11 0.32 0.5983720930232559
0.11 0.33 0.6100000000000001
0.11 0.34 0.6211111111111112
0.11 0.35 0.6317391304347826
0.11 0.36 0.6419148936170213
0.11 0.37 0.6516666666666667
0.11 0.38 0.6610204081632653
0.11 0.39 0.67
0.11 0.4 0.6786274509803922
0.11 0.41 0.6869230769230769
0.11 0.42 0.6949056603773585
0.11 0.43 0.7025925925925925
0.11 0.44 0.71
0.11 0.45 0.7171428571428571
0.11 0.46 0.7240350877192983
0.11 0.47 0.7306896551724138
0.11 0.48 0.7371186440677966
0.11 0.49 0.7433333333333334
0.11 0.5 0.749344262295082
0.11 0.51 0.7551612903225807
0.11 0.52 0.7607936507936508
0.11 0.53 0.76625
0.11 0.54

0.16 0.28 0.43272727272727274
0.16 0.29 0.4488888888888889
0.16 0.3 0.46434782608695657
0.16 0.31 0.4791489361702128
0.16 0.32 0.4933333333333334
0.16 0.33 0.5069387755102042
0.16 0.34 0.52
0.16 0.35 0.5325490196078431
0.16 0.36 0.5446153846153846
0.16 0.37 0.5562264150943396
0.16 0.38 0.5674074074074074
0.16 0.39 0.5781818181818181
0.16 0.4 0.5885714285714285
0.16 0.41 0.5985964912280701
0.16 0.42 0.6082758620689656
0.16 0.43 0.6176271186440678
0.16 0.44 0.6266666666666667
0.16 0.45 0.6354098360655739
0.16 0.46 0.6438709677419355
0.16 0.47 0.652063492063492
0.16 0.48 0.6599999999999999
0.16 0.49 0.6676923076923077
0.16 0.5 0.6751515151515151
0.16 0.51 0.6823880597014925
0.16 0.52 0.6894117647058823
0.16 0.53 0.696231884057971
0.16 0.54 0.7028571428571428
0.16 0.55 0.7092957746478873
0.16 0.56 0.7155555555555555
0.16 0.57 0.7216438356164383
0.16 0.58 0.7275675675675675
0.16 0.59 0.7333333333333333
0.16 0.6 0.7389473684210526
0.16 0.61 0.7444155844155844
0.16 0.62 0.7497435897435897
0.1

0.21 0.1 -0.14483870967741933
0.21 0.11 -0.10249999999999995
0.21 0.12 -0.06272727272727277
0.21 0.13 -0.025294117647058828
0.21 0.14 0.010000000000000037
0.21 0.15 0.043333333333333335
0.21 0.16 0.07486486486486488
0.21 0.17 0.1047368421052632
0.21 0.18 0.1330769230769231
0.21 0.19 0.16000000000000003
0.21 0.2 0.18560975609756103
0.21 0.21 0.21
0.21 0.22 0.23325581395348838
0.21 0.23 0.2554545454545455
0.21 0.24 0.27666666666666667
0.21 0.25 0.29695652173913045
0.21 0.26 0.31638297872340426
0.21 0.27 0.3350000000000001
0.21 0.28 0.3528571428571429
0.21 0.29 0.37
0.21 0.3 0.3864705882352941
0.21 0.31 0.4023076923076923
0.21 0.32 0.4175471698113208
0.21 0.33 0.43222222222222223
0.21 0.34 0.4463636363636364
0.21 0.35 0.45999999999999996
0.21 0.36 0.4731578947368421
0.21 0.37 0.4858620689655173
0.21 0.38 0.49813559322033896
0.21 0.39 0.51
0.21 0.4 0.5214754098360657
0.21 0.41 0.5325806451612903
0.21 0.42 0.5433333333333333
0.21 0.43 0.55375
0.21 0.44 0.5638461538461539
0.21 0.45 0.5736363

...

0.79 0.84 0.8206748466257668
0.79 0.85 0.8265853658536585
0.79 0.86 0.8324242424242424
0.79 0.87 0.8381927710843373
0.79 0.88 0.8438922155688623
0.79 0.89 0.8495238095238096
0.79 0.9 0.8550887573964497
0.79 0.91 0.8605882352941177
0.79 0.92 0.8660233918128656
0.79 0.93 0.8713953488372094
0.79 0.94 0.8767052023121387
0.79 0.95 0.8819540229885058
0.79 0.96 0.8871428571428571
0.79 0.97 0.8922727272727273
0.79 0.98 0.8973446327683616
0.79 0.99 0.9023595505617977
0.79 1.0 0.9073184357541899
0.8 0.01 -0.1753086419753086
0.8 0.02 -0.15121951219512186
0.8 0.03 -0.12771084337349392
0.8 0.04 -0.10476190476190461
0.8 0.05 -0.0823529411764704
0.8 0.06 -0.060465116279069586
0.8 0.07 -0.03908045977011476
0.8 0.08 -0.018181818181818188
0.8 0.09 0.0022471910112359383
0.8 0.1 0.022222222222222254
0.8 0.11 0.04175824175824172
0.8 0.12 0.060869565217391286
0.8 0.13 0.0795698924731183
0.8 0.14 0.09787234042553195
0.8 0.15 0.11578947368421055
0.8 0.16 0.13333333333333341
0.8 0.17 0.1505154639175259
0.8 0.1

0.84 0.87 0.8575438596491228
0.84 0.88 0.8632558139534884
0.84 0.89 0.8689017341040463
0.84 0.9 0.8744827586206897
0.84 0.91 0.88
0.84 0.92 0.8854545454545455
0.84 0.93 0.8908474576271187
0.84 0.94 0.8961797752808989
0.84 0.95 0.9014525139664804
0.84 0.96 0.9066666666666666
0.84 0.97 0.9118232044198895
0.84 0.98 0.916923076923077
0.84 0.99 0.921967213114754
0.84 1.0 0.9269565217391305
0.85 0.01 -0.1267441860465116
0.85 0.02 -0.1040229885057471
0.85 0.03 -0.08181818181818179
0.85 0.04 -0.06011235955056171
0.85 0.05 -0.03888888888888886
0.85 0.06 -0.01813186813186829
0.85 0.07 0.0021739130434781373
0.85 0.08 0.022043010752688108
0.85 0.09 0.04148936170212758
0.85 0.1 0.06052631578947365
0.85 0.11 0.07916666666666661
0.85 0.12 0.09742268041237112
0.85 0.13 0.11530612244897953
0.85 0.14 0.13282828282828285
0.85 0.15 0.15000000000000002
0.85 0.16 0.16683168316831687
0.85 0.17 0.18333333333333335
0.85 0.18 0.19951456310679616
0.85 0.19 0.2153846153846155
0.85 0.2 0.23095238095238102
0.85 0.2

0.89 0.83 0.8551162790697674
0.89 0.84 0.8610982658959537
0.89 0.85 0.8670114942528736
0.89 0.86 0.8728571428571429
0.89 0.87 0.8786363636363637
0.89 0.88 0.8843502824858757
0.89 0.89 0.89
0.89 0.9 0.895586592178771
0.89 0.91 0.9011111111111111
0.89 0.92 0.9065745856353591
0.89 0.93 0.911978021978022
0.89 0.94 0.9173224043715846
0.89 0.95 0.9226086956521738
0.89 0.96 0.9278378378378378
0.89 0.97 0.933010752688172
0.89 0.98 0.9381283422459893
0.89 0.99 0.9431914893617022
0.89 1.0 0.9482010582010582
0.9 0.01 -0.07802197802197797
0.9 0.02 -0.05652173913043468
0.9 0.03 -0.03548387096774186
0.9 0.04 -0.01489361702127645
0.9 0.05 0.0052631578947369695
0.9 0.06 0.02499999999999991
0.9 0.07 0.04432989690721645
0.9 0.08 0.06326530612244896
0.9 0.09 0.08181818181818179
0.9 0.1 0.09999999999999998
0.9 0.11 0.11782178217821782
0.9 0.12 0.1352941176470588
0.9 0.13 0.15242718446601944
0.9 0.14 0.1692307692307693
0.9 0.15 0.18571428571428572
0.9 0.16 0.20188679245283025
0.9 0.17 0.21775700934579445
0

0.94 0.85 0.8897206703910614
0.94 0.86 0.8955555555555555
0.94 0.87 0.9013259668508287
0.94 0.88 0.907032967032967
0.94 0.89 0.9126775956284153
0.94 0.9 0.9182608695652174
0.94 0.91 0.9237837837837838
0.94 0.92 0.929247311827957
0.94 0.93 0.9346524064171123
0.94 0.94 0.94
0.94 0.95 0.9452910052910053
0.94 0.96 0.9505263157894737
0.94 0.97 0.9557068062827224
0.94 0.98 0.9608333333333333
0.94 0.99 0.9659067357512953
0.94 1.0 0.9709278350515463
0.95 0.01 -0.029166666666666674
0.95 0.02 -0.008762886597938113
0.95 0.03 0.011224489795918391
0.95 0.04 0.03080808080808084
0.95 0.05 0.050000000000000044
0.95 0.06 0.06881188118811887
0.95 0.07 0.08725490196078434
0.95 0.08 0.10533980582524272
0.95 0.09 0.12307692307692308
0.95 0.1 0.14047619047619053
0.95 0.11 0.15754716981132078
0.95 0.12 0.17429906542056062
0.95 0.13 0.19074074074074077
0.95 0.14 0.206880733944954
0.95 0.15 0.22272727272727266
0.95 0.16 0.23828828828828819
0.95 0.17 0.25357142857142856
0.95 0.18 0.2685840707964601
0.95 0.19 0.

0.99 0.99 0.99
0.99 1.0 0.9950251256281407
1.0 0.01 0.01980198019801982
1.0 0.02 0.03921568627450989
1.0 0.03 0.058252427184466105
1.0 0.04 0.07692307692307698
1.0 0.05 0.09523809523809534
1.0 0.06 0.1132075471698114
1.0 0.07 0.13084112149532723
1.0 0.08 0.14814814814814814
1.0 0.09 0.1651376146788991
1.0 0.1 0.18181818181818188
1.0 0.11 0.19819819819819828
1.0 0.12 0.2142857142857143
1.0 0.13 0.23008849557522115
1.0 0.14 0.2456140350877194
1.0 0.15 0.26086956521739124
1.0 0.16 0.27586206896551724
1.0 0.17 0.29059829059829057
1.0 0.18 0.30508474576271183
1.0 0.19 0.31932773109243695
1.0 0.2 0.33333333333333326
1.0 0.21 0.3471074380165289
1.0 0.22 0.360655737704918
1.0 0.23 0.3739837398373983
1.0 0.24 0.3870967741935484
1.0 0.25 0.4
1.0 0.26 0.4126984126984127
1.0 0.27 0.4251968503937008
1.0 0.28 0.4375
1.0 0.29 0.4496124031007752
1.0 0.3 0.46153846153846156
1.0 0.31 0.4732824427480916
1.0 0.32 0.484848484848485
1.0 0.33 0.4962406015037595
1.0 0.34 0.5074626865671643
1.0 0.35 0.51851851